## Importing important libraries

#### Steps to be followed

* Importing necessary Libraries
* Creating S3 bucket
* Mapping train and test data in S3
* Mapping the path of the models in S3

In [53]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

In [54]:
bucket_name = 'bankapplication01'# <-- change according to your selection
my_region = boto3.session.Session().region_name #<-----set the region of the instance
print(my_region)

us-east-1


In [55]:
s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket has been created successfully')
except Exception as e:
    print('error :', e)

S3 bucket has been created successfully


In [56]:
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bankapplication01/xgboost-as-a-built-in-algo/output


## Download the Dataset ans Storing in S3

In [57]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Successfully downloaded')
except Exception as e:
    print('Data load error:',e)
    
try: 
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data öpaded into dataframe')
except Exception as e:
    print('Data load error:', e)

Successfully downloaded
Success: Data öpaded into dataframe


In [58]:
#### Train-Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=14432), [int(0.7*len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [59]:
## Saving Train and Test into Buckets
# We start with Train Data
import os
pd.concat([train_data['y_yes'],train_data.drop(['y_no','y_yes'],axis=1)],axis=1).to_csv('train.csv',index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name,prefix), content_type='csv')

In [60]:
pd.concat([train_data['y_yes'],train_data.drop(['y_no','y_yes'],axis=1)],axis=1).to_csv('test.csv',index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name,prefix), content_type='csv')

## Building Models XgBoost-inbuilt Algorithm

In [61]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
from sagemaker import image_uris
#container = image_uris.retrieve(boto3.Session().region_name,
#                          framework='xgboost')
#container = image_uris.retrieve(region=boto3.Session().region_name, framework="xgboost")
container = sagemaker.image_uris.retrieve("xgboost", sagemaker.Session().boto_region_name, "latest")
display(container)

'811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest'

In [62]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [68]:
# Construct a sagemaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container,
                                         hyperparameters = hyperparameters,
                                         role=sagemaker.get_execution_role(),
                                         train_instance_count=1,
                                         train_instance_type='ml.m5.2xlarge',
                                         train_volume_size=5,
                                         output_path=output_path,
                                         train_use_spot_instances=True,
                                         train_max_run=50,
                                         train_max_wait=500)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [69]:
estimator.fit({'train':s3_input_train, 'validation':s3_input_test})

2022-04-07 14:58:06 Starting - Starting the training job...
2022-04-07 14:58:33 Starting - Launching requested ML instancesProfilerReport-1649343486: InProgress
.........
2022-04-07 14:59:53 Starting - Preparing the instances for training......
2022-04-07 15:01:01 Downloading - Downloading input data...
2022-04-07 15:01:34 Training - Training image download completed. Training in progress.
2022-04-07 15:01:34 Uploading - Uploading generated training modelArguments: train
[2022-04-07:15:01:25:INFO] Running standalone xgboost training.
[2022-04-07:15:01:25:INFO] File size need to be processed in the node: 10.14mb. Available memory size in the node: 23747.66mb
[2022-04-07:15:01:25:INFO] Determined delimiter of CSV input is ','
[15:01:25] S3DistributionType set as FullyReplicated
[15:01:25] 57662x59 matrix with 3402058 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2022-04-07:15:01:25:INFO] Determined delimiter of CSV input is ','
[15:01:25] S3Distribut

In [70]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

--------------!

In [75]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
#xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


(12357,)


In [76]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 90.0%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10823)    33% (151)
Purchase        9% (1081)     67% (302) 



#### deleting the endpoints

In [78]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "arn:aws:sagemaker:us-east-1:805810228251:endpoint/xgboost-2022-04-07-15-06-09-631".